<a href="https://colab.research.google.com/github/njainds/NLP_kaggle_GoogleQuest/blob/master/Baseline_Iteration_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Google_Quest"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
datasets


In [0]:
#"bert-base-uncased": "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin"
#"bert-base-uncased": "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json",
#https://www.kaggle.com/phoenix9032/pytorch-bert-plain/data
#https://huggingface.co/transformers/model_doc/bert.html?highlight=bertmodel#transformers.BertModel

In [4]:
import os
os.chdir("/content/drive/My Drive/Google_Quest")
os.getcwd()
!ls

datasets


In [5]:
## Loading Libraries  -No they are not for books
!pip install transformers
import transformers, sys, os, gc
import numpy as np, pandas as pd, math
import torch, random, os, multiprocessing, glob
import torch.nn.functional as F
import torch, time

from datasets.helpers.ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from scipy.stats import spearmanr
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset,RandomSampler, SequentialSampler
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)
from transformers.modeling_bert import BertPreTrainedModel 
from tqdm import tqdm
print(transformers.__version__)

     |████████████████████████████████| 542kB 4.7MB/s 
     |████████████████████████████████| 1.0MB 20.6MB/s 
     |████████████████████████████████| 3.7MB 34.4MB/s 
     |████████████████████████████████| 870kB 66.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=cf32fc40db34115f176fe6defe31d4ccc4091e3a7a1cc98c2179e55831a31287
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


2.6.0


In [0]:
ROOT = 'datasets/google-quest-challenge/'
## Make results reproducible .Else noone will believe you .
import random

def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [0]:
class PipeLineConfig:
    def __init__(self, lr, warmup,accum_steps, epochs, seed, expname,head_tail,freeze,question_weight,answer_weight,fold,train):
        self.lr = lr
        self.warmup = warmup
        self.accum_steps = accum_steps
        self.epochs = epochs
        self.seed = seed
        self.expname = expname
        self.head_tail = head_tail
        self.freeze = freeze
        self.question_weight = question_weight
        self.answer_weight =answer_weight
        self.fold = fold
        self.train = train

In [0]:
config_1 = PipeLineConfig(3e-5,0.05,4,0,42,'uncased_1',True,False,0.7,0.3,8,False)
config = config_1


In [0]:
seed_everything(config.seed)

In [10]:
## load the data 
train = pd.read_csv(ROOT+'train.csv')
test = pd.read_csv(ROOT+'test.csv')
sub = pd.read_csv(ROOT+'sample_submission.csv')
train_len, test_len ,sub_len = len(train.index), len(test.index),len(sub.index)
print(f'train size: {train_len}, test size: {test_len} , sample size: {sub_len}')

train size: 6079, test size: 476 , sample size: 476


In [11]:
target_cols = sub.columns.tolist()[1:]
len(target_cols)

30

In [0]:
# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=290, t_max_len=30, q_max_len=128, a_max_len=128):
    
    #350+128+30 = 508 + 4 = 512
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        q_len_head = round(q_new_len/2)
        q_len_tail = -1* (q_new_len -q_len_head)
        a_len_head = round(a_new_len/2)
        a_len_tail = -1* (a_new_len -a_len_head)        ## Head+Tail method .
        t = t[:t_new_len]
        if config.head_tail :
            q = q[:q_len_head]+q[q_len_tail:]
            a = a[:a_len_head]+a[a_len_tail:]
        else:
            q = q[:q_new_len]
            a = a[:a_new_len] ## No Head+Tail ,usual processing
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
   # stoken = ["[CLS]"] + title  + question  + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def _get_stoken_output(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
    return stoken

def compute_input_tokens(df, columns, tokenizer, max_sequence_length):
    
    input_tokens, input_masks, input_segments = [], [], []
    for _, instance in df[columns].iterrows():
        t, q, a = instance.question_title, instance.question_body, instance.answer
        t, q, a = _trim_input(t, q, a, max_sequence_length)
        tokens= _get_stoken_output(t, q, a, tokenizer, max_sequence_length)
        input_tokens.append(tokens)
    return input_tokens

def compute_input_arays(df, columns, tokenizer, max_sequence_length,t_max_len=30, q_max_len=128, a_max_len=128):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in df[columns].iterrows():
        t, q, a = instance.question_title, instance.question_body, instance.answer
        t, q, a = _trim_input(t, q, a, max_sequence_length,t_max_len, q_max_len, a_max_len)
        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])


In [0]:
class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, lengths, labels = None):
        
        self.inputs = inputs
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        self.lengths = lengths

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        lengths         = self.lengths[idx]
        if self.labels is not None: # targets
            labels = self.labels[idx]
            return input_ids, input_masks, input_segments, labels, lengths
        return input_ids, input_masks, input_segments, lengths

    def __len__(self):
        return len(self.inputs[0])

In [0]:
class CustomBert(BertPreTrainedModel):

    def __init__(self, config):
        super(CustomBert, self).__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.bn = nn.BatchNorm1d(1024)
        self.linear  = nn.Linear(config.hidden_size,1024)
        self.classifier = nn.Linear(1024, self.config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        lin_output = F.relu(self.bn(self.linear(pooled_output))) ## Note : This Linear layer is added without expert supervision . This will worsen the results . 
                                               ## But you are smarter than me , so you will figure out,how to customize better.
        lin_output = self.dropout(lin_output)    
        logits = self.classifier(lin_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [15]:
#test
from transformers import BertModel, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True, output_attentions=True)

input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)

#outputs[3][12].shape
len(outputs) #4
outputs[0].shape #torch.Size([1, 8, 768])
outputs[1].shape #torch.Size([1, 768])
len(outputs[2]) #tuple of 13 elements each of torch.Size([1, 768])
len(outputs[3]) #tuple of 12 elements each of torch.Size([1, 12, 8, 8])

#format of output: ((tensor),(tensor),((tensor)*13),((tensor)*12))


12

In [0]:
# Train fct

def train_model(train_loader, optimizer, criterion, scheduler,  config):
  model.train()
  avg_losst = 0
  avg_loss1 = 0
  avg_loss2 = 0
  avg_loss = 0

  optimizer.zero_grad()
  for idx, batch in enumerate(train_loader):
    input_ids, input_masks, input_segments, labels, _ = batch
    input_ids, input_masks, input_segments, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), labels.to(device)
    outputs = model(input_ids = input_ids.long(), attention_masks = input_masks, token_type_ids = input_segments, labels=None)
    logits = outputs[0]
    tot_loss = criterion(logits, labels)
    q_loss = criterion(logits[:21], labels[:21])
    a_loss = criterion(logits[21:], labels[21:])
    loss = config.question_weight*q_loss + config.answer_weight*a_loss
    loss.backward()
    if (idx+1) % config.accum_steps ==0:
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()

    avg_losst += tot_loss.item()/len(train_loader)
    avg_loss1 += q_loss.item()/len(train_loader)
    avg_loss2 += a_loss.item()/len(train_loader)
    avg_loss += loss.item()/len(train_loader)
    del input_ids, input_masks, input_segments, labels
  
  torch.cuda.empty_cache()
  gc.collect()
  return avg_losst, avg_loss1, avg_loss2, avg_loss


In [0]:
def val_model(val_loader, val_shape, batch_size=8):
  avg_val_loss = 0
  model.eval()
  val_preds = np.zeros((val_shape,len(target_cols)))  
  originals = np.zeros((val_shape,len(target_cols)))
  with torch.no_grad():
    for idx, batch in enumerate(val_loader):
      input_ids, input_masks, input_segments, labels, _ = batch
      val_outputs = model(input_ids = input_ids.long(), attention_masks = input_masks, token_type_ids = input_segments, labels=None)
      logits = val_outputs[0]
      avg_val_loss += criterion(logits, labels).item()/len(val_loader)
      val_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
      originals[idx*batch_size : (idx+1)*batch_size] = labels.detach().cpu().squeeze().numpy()

    score = 0
    preds = torch.sigmoid(torch.tensor(val_preds)).numpy()
    rho_val = np.mean([spearmanr(original[:i],preds[:i]).correlation for i in range(preds.shape[1])])
    print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
  return avg_val_loss, rho_val


In [0]:
# Predict

def predict_result(model, test_loader, batch_size):
  model.eval()
  preds = np.zeros((len(test), len(target_cols)))

  tz = tqdm(enumerate(test_loader))
  for idx, batch in tz:
    with torch.no_grad():
      test_out = model(inputs_ids = batch[0].to(device), attention_masks = batch[1].to(device), labels = None, token_type_ids = batch[2].to(device))
      test_out = test_out[0]
      preds[idx*batch_size : (idx+1)*batch_size]  = test_out.detach().cpu().squeeze().numpy()
  
  output  = torch.sigmoid(torch.tensor(preds)).numpy()
  return output


In [19]:
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

0
1
Tesla P100-PCIE-16GB
True


In [25]:
# Prepare inputs for model training

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
input_categories = list(train.columns[[1,2,5]])
input_categories

bert_config = BertConfig.from_json_file('./datasets/transformers/bert-base-uncased-config.json')
bert_config.num_labels = len(target_cols)
bert_config.output_attentions = True
bert_config.output_hidden_states = True

bert_model = 'bert-base-uncased'
do_lower_case = 'uncased' in bert_model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
output_model_file = 'plain_pytorch_model1.bin'

test_inputs = compute_input_arays(test, input_categories, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=239, a_max_len=239)
lengths_test = np.argmax(test_inputs[0] == 0, axis=1)
lengths_test[lengths_test == 0] = test_inputs[0].shape[1]

test_set = QuestDataset(test_inputs, lengths_test, labels=None)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
results = np.zeros((len(test),len(target_cols)))

print(results.shape)
print(len(test_loader))

(476, 30)
15


In [21]:
# Train the model and evaluate

num_folds = config.fold
seed  = config.seed
batch_size = 8
epochs = config.epochs
accum_steps  = 1

kf = MultilabelStratifiedKFold(n_splits = num_folds, random_state = seed)
y_train = train[target_cols].values

if config.train:
  for fold, (train_idx, val_idx) in enumerate(kf.split(train.values, y_train)):
    if fold>0:
      break
    print("Current fold is : ", fold)
    
    train_df, val_df = train.iloc[train_idx], train.iloc[val_idx]
    print("train shape is %s and val shape is %s" % (train_df.shape, val_df.shape))
    
    inputs_train = compute_input_arays(train_df, input_categories, max_sequence_length=512, t_max_len=30, q_max_len=239,a_max_len=239)
    outputs_train = compute_output_arrays(train_df, columns=target_cols)
    outputs_train = torch.tensor(outputs_train,dtype=torch.float32)
    lengths_train = np.argmax(inputs_train[0] == 0, axis=1)
    lengths_train[lengths_train == 0] = inputs_train[0].shape[1] 
    train_set = QuestDataset(inputs_train, lenghts_train, labels = outputs_train)
    train_sampler = RandomSampler(train_set)
    train_loader = DataLoader(train_set, batch_size = batch_size, sampler=train_sampler)

    inputs_val = compute_input_arays(val_df, input_categories, max_sequence_length=512, t_max_len=30, q_max_len=239,a_max_len=239)
    outputs_val = compute_output_arrays(val_df, columns=target_cols)
    outputs_val = torch.tensor(outputs_val,dtype=torch.float32)
    lengths_val = np.argmax(inputs_val[0] == 0, axis=1)
    lengths_val[lengths_val == 0] = inputs_val[0].shape[1] 
    val_set = QuestDataset(inputs_val, lenghts_val, labels = outputs_val)
    val_loader = DataLoader(val_set, batch_size = batch_size, shuffle=False, drop_last=False)

    model = CustomBert.from_pretrained(bert_model, config = bert_config)
    model.zero_grad()
    model.to(device)
    torch.cuda.empty_cache()
    
    if config.freeze:
      for param in model.bert.parameters():
        param.requires_grad = False
    model.train()

    i=0
    best_avg_loss = 100
    best_score = -1
    best_param_loss = None
    best_param_score = None

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias','LayerNorm.bias','LayerNorm.weight']
    optimizer_grouped_parameters = [
                                    {'params':[p for n,p in param_optimizer if not any(nd in n for nd in no_decay)],
                                     'weight_decay': 0.8
                                     },
                                    {'params':[p for n,p in param_optimizer if any(nd in n for nd in no_decay)],
                                     'weight_decay': 0
                                     }
                                    ]
    optimizer = torch.optim.AdamW(optimizer.optimizer_grouped_parameters, lr = config.lr, eps = 4e-5)
    criterion = nn.BCEWithLogitsLoss()
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps=epochs*len(train_loader)//accum_steps)
    print("##### Training ######")
    
    for epoch in range(epochs):
      torch.cuda.empty_cache()
      start_time = time.time()
      avg_losst, avg_loss1, avg_loss2, avg_loss = train_model(train_loader, optimizer, criterion, scheduler, config)
      avg_val_loss, score = val_model(val_loader, val_df.shape[0])
      elapsed_time = time.time() - start_time
      print("Epoch {}/{} \t
       train_loss_overall = {:.4f} \t train_loss = {:.4f} \t train_loss_Question = {:.4f} \t train_loss_Answer = {:.4f} \t
       val_loss_overall = {:.4f} \t score={:.6f} \t time = {:.2f}s".format(epoch+1, epochs, avg_losst,avg_loss, avg_loss1, avg_loss2, 
                                                         avg_val_loss,score, elapsed_time))
      if best_avg_loss>avg_val_loss:
        i=0
        best_avg_loss = avg_val_loss
        best_param_loss = model.state_dict()

      if best_score < score
      best_score = score
      best_param_score = model.state_dict()
      print("best_param_score_{}_{}.pt".format(config.expname, fold+1))
      torch.save(best_param_score, "best_param_score_{}_{}.pt".format(config.expname, fold+1))
      else:
        i=i+1

    model.load_state_dict(best_param_score)
    result += predict_result(model, test_loader)/num_folds

  del train_df, val_df, model, optimizer, scheduler, criterion, val_loader, train_loader, test_loader, val_set, train_set
  torch.cuda.empty_cache()
  gc.collect()   



print("Final result: %s", %(result.shape))



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
submission = pd.read_csv(r'../input/google-quest-challenge/sample_submission.csv')
submission.loc[:, 'question_asker_intent_understanding':] = result
#submission.loc[~submission['qa_id'].isin(qa_id_list),'question_type_spelling']=0.0
#submission.loc[submission['qa_id'].isin(qa_id_list),'question_type_spelling'] = 1.0

submission.to_csv('submission.csv', index=False)
submission.head()